In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
%%HTML
<style type="text/css">
table.dataframe td, table.dataframe th {
    border: 1px  black solid !important;
  color: black !important;
}
</style>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm

from matplotlib import rcParams
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
import folium as fo
import json, requests

#Load data sets in to dataframes
patients_data_df = pd.read_csv(r'/kaggle/input/covid19-corona-virus-india-dataset/patients_data.csv')
patients_data_df.head()

In [ ]:
#count cases based on states
patients_statewise=patients_data_df.groupby(['detected_state'])['p_id'].count().to_frame(name='Number of patients').reset_index()
patients_statewise

#get foreigners cases based on their nationality
foreigners_cases=patients_data_df.groupby(['p_id','nationality','detected_state','current_status'])['p_id'].count().to_frame(name='Number of patients').reset_index()
foreigners_cases = foreigners_cases[foreigners_cases.nationality !="India" ]
foreigners_cases

foreigners_status_count=foreigners_cases.groupby(['detected_state','current_status'])['p_id'].count().to_frame(name='Number of patients').reset_index()
#Pivot table and remove duplicate indexes
foreigners_status_count=foreigners_status_count.pivot(index='detected_state',columns='current_status', values='Number of patients')
foreigners_status_count

In [ ]:
locations = {
    "Kerala" : [10.8505,76.2711],
    "Maharashtra" : [19.7515,75.7139],
    "Karnataka": [15.3173,75.7139],
    "Telangana": [18.1124,79.0193],
    "Uttar Pradesh": [26.8467,80.9462],
    "Rajasthan": [27.0238,74.2179],
    "Gujarat":[22.2587,71.1924],
    "Delhi" : [28.7041,77.1025],
    "Punjab":[31.1471,75.3412],
    "Tamil Nadu": [11.1271,78.6569],
    "Haryana": [29.0588,76.0856],
    "Madhya Pradesh":[22.9734,78.6569],
    "Jammu and Kashmir":[33.7782,76.5762],
    "Ladakh": [34.1526,77.5770],
    "Andhra Pradesh":[15.9129,79.7400],
    "West Bengal": [22.9868,87.8550],
    "Bihar": [25.0961,85.3131],
    "Chhattisgarh":[21.2787,81.8661],
    "Chandigarh":[30.7333,76.7794],
    "Uttarakhand":[30.0668,79.0193],
    "Himachal Pradesh":[31.1048,77.1734],
    "Goa": [15.2993,74.1240],
    "Odisha":[20.9517,85.0985],
    "Andaman and Nicobar Islands": [11.7401,92.6586],
    "Puducherry":[11.9416,79.8083],
    "Manipur":[24.6637,93.9063],
    "Mizoram":[23.1645,92.9376],
    "Assam":[26.2006,92.9376],
    "Meghalaya":[25.4670,91.3662],
    "Tripura":[23.9408,91.9882],
    "Arunachal Pradesh":[28.2180,94.7278],
    "Jharkhand" : [23.6102,85.2799],
    "Nagaland": [26.1584,94.5624],
    "Sikkim": [27.5330,88.5122],
    "Dadra and Nagar Haveli":[20.1809,73.0169],
    "Lakshadweep":[10.5667,72.6417],
    "Daman and Diu":[20.4283,72.8397]    
}

foreigners_status_count["Lat"] = ""
foreigners_status_count["Long"] = ""

#foreigners_status_count
#Add latitude and longitude to each of the states
for index in foreigners_status_count.index :
    foreigners_status_count.loc[foreigners_status_count.index == index,"Lat"] = locations[index][0]
    foreigners_status_count.loc[foreigners_status_count.index == index,"Long"] = locations[index][1]

#remove NAN as zero

foreigners_status_count.fillna(0, inplace=True)
sum_column = foreigners_status_count["Deceased"] + foreigners_status_count["Hospitalized"]+ foreigners_status_count["Recovered"]
foreigners_status_count["Total"] = sum_column
foreigners_status_count


In [ ]:
india = fo.Map(title="Your map title",location=[23,80], zoom_start=4,max_zoom=6,min_zoom=4,height=500,width="80%")
for i in range(0,len(foreigners_status_count.index)):
    fo.Circle(
        location=[foreigners_status_count.iloc[i]['Lat'], foreigners_status_count.iloc[i]['Long']],
        tooltip = "<h5 style='text-align:center;font-weight: bold'>"+foreigners_status_count.iloc[i].name+"</h5>"+
                    "<hr style='margin:10px;'>"+
                    "<ul style='color: #444;list-style-type:circle;align-item:left;padding-left:20px;padding-right:20px'>"+
        "<li>No. of Deceased patients: "+str(foreigners_status_count.iloc[i]['Deceased'])+"</li>"+
        "<li>No. of Hospitalized patients:   "+str(foreigners_status_count.iloc[i]['Hospitalized'])+"</li>"+
        "<li>No. of Recovered patients:   "+str(foreigners_status_count.iloc[i]['Recovered'])+"</li>"+
        "</ul>"
        ,
        radius=(int(np.log(foreigners_status_count.iloc[i]['Total']*30)))*20000,
        color='#ff6600',
        fill_color='#ff8533',
        fill=True).add_to(india)

india
